In [1]:
import turicreate

In [2]:
song_data = turicreate.SFrame('~/venv/ml_wash/ml_wash_course1/song_data/song_data.sframe/')

In [3]:
# song_data - count unique artists columns
kanye_t = turicreate.SFrame.filter_by(song_data,'Kanye West','artist',exclude=False)
foo_t = turicreate.SFrame.filter_by(song_data,'Foo Fighters','artist',exclude=False)
tay_t = turicreate.SFrame.filter_by(song_data,'Taylor Swift','artist',exclude=False)
gaga_t = turicreate.SFrame.filter_by(song_data,'Lady GaGa','artist',exclude=False)

In [4]:
# calculate number of unique users for the various artists
kanye_users = kanye_t['user_id'].unique()
foo_users = foo_t['user_id'].unique()
tay_users = tay_t['user_id'].unique()
gaga_users = gaga_t['user_id'].unique()

In [5]:
# print unique artists for kanye
len(kanye_users)

2522

In [6]:
# print unique artists for foo fighters
len(foo_users)

2055

In [7]:
# print unique artists for taylor swift
len(tay_users)

3246

In [8]:
# print unique artists for lady gaga
len(gaga_users)

2928

In [9]:
# new column listing total count of each artist - most popular 
song_data.groupby(key_column_names='artist', operations={'total_count': turicreate.aggregate.SUM('listen_count')}).sort('total_count',ascending=False)

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [10]:
# sort by least popular
song_data.groupby(key_column_names='artist', operations={'total_count': turicreate.aggregate.SUM('listen_count')}).sort('total_count',ascending=True)

artist,total_count
William Tabbert,14
Reel Feelings,24
Beyoncé feat. Bun B andSlim Thug ...,26
Diplo,30
Boggle Karaoke,30
harvey summers,31
Nâdiya,36
Kanye West / Talib Kweli/ Q-Tip / Common / ...,38
Jody Bernal,38
Aneta Langerova,38


# optional - use group aggregate to find most recommended songs

In [11]:
# 80% training, 20% testing, seed 0
train_data,test_data = song_data.random_split(.8,seed=0)

In [12]:
# train the item_similarity_recommender
popularity_model = turicreate.popularity_recommender.create(train_data,
                                                           user_id = 'user_id',
                                                           item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

Data prepared in: 0.429899s

893580 observations to process; with 9952 unique items.

In [13]:
# only use first 10,000 users (for speed purposes)
subset_test_users = test_data['user_id'].unique()[0:10000]

In [14]:
# compute recommendations
personalized_model = popularity_model.recommend(subset_test_users,k=1)

recommendations finished on 1000/10000 queries. users per second: 47478.9

recommendations finished on 2000/10000 queries. users per second: 58166.6

recommendations finished on 3000/10000 queries. users per second: 63116.7

recommendations finished on 4000/10000 queries. users per second: 65624.3

recommendations finished on 5000/10000 queries. users per second: 67102.4

recommendations finished on 6000/10000 queries. users per second: 68206.6

recommendations finished on 7000/10000 queries. users per second: 69030.1

recommendations finished on 8000/10000 queries. users per second: 69648.8

recommendations finished on 9000/10000 queries. users per second: 70173.8

recommendations finished on 10000/10000 queries. users per second: 70059.4

In [15]:
# count how often each song is recommended. Not sure if this is correct
personalized_model.groupby(key_column_names='song',operations={'count': turicreate.aggregate.COUNT()}).sort('count',ascending=False)

song,count
Sehr kosmisch - Harmonia,9230
Undo - Björk,634
You're The One - DwightYoakam ...,70
Dog Days Are Over (RadioEdit) - Florence + The ...,29
Horn Concerto No. 4 in Eflat K495: II. Romance ...,13
Revelry - Kings Of Leon,10
Hey_ Soul Sister - Train,6
Secrets - OneRepublic,5
Fireflies - CharttraxxKaraoke ...,2
OMG - Usher featuringwill.i.am ...,1
